In [8]:
import numpy as np
import pickle

In [19]:
class Dataset():
    TRAINING_IMG_FILENAME = 'train_image.pkl' 
    TRAINING_LABEL_FILENAME = 'train_label.pkl'
    TESTING_IMG_FILENAME = 'test_image.pkl'
    
    def __init__(self):
        self.training_images = self.load_pickle(self.TRAINING_IMG_FILENAME)
        self.training_labels = self.load_pickle(self.TRAINING_LABEL_FILENAME)
        self.test_images = self.load_pickle(self.TESTING_IMG_FILENAME)
    
    def load_pickle(self, filename, mode='rb'):
        return np.array(pickle.load(open(filename, mode)))